This code was used to model the initial teen pregnancy dataset with all pulled features. The second notebook file is the same modeling process but on fewer, more important features

In [252]:
### Importing all of the cool things I'll need...

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, cross_val_score, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.learning_curve import learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier

%matplotlib inline

In [ ]:
### Function for calculating cross-validation scores across models

def cv_score_means(model, X, y, score_type):
    scores = cross_val_score(model, X, y, cv=30, scoring=score_type)
    return scores.mean()

In [1]:
### Increases the display width of the Jupyter notebook

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [254]:
### Reads the feature file to a pandas dataframe

df = pd.read_csv('mcnulty.csv')

In [257]:
### Feature list for Random Forest feature importance measure
### Five columns of random noise are added to test features

rng = [x for x in range(1, 6)]

for i in rng:
    col_name = 'RAND_' + str(i)
    df[col_name] = 1
    df[col_name] = df[col_name].map(lambda x: x * random.randint(0, 100))


feature_list = ['% Smokers', '% Obese', '% Binge Drinking',
       'Chlamydia Rates per 100,000', 'Primary Care Physician Rate',
       'No of Medicare enrollees', 'Avg Freshman Graduation Rate', '% College',
       '% unemployed', '% Children in Poverty', 'GINI',
       '% Single-Parent Households', 'Violent Crime Rate', 'Liquor Store Rate',
       'Adherents %', 'Congregations Per 10K People', 'Hispanic',
       'Hispanic, younger than 20', 'Non-Hipanic Black, younger than 20',
       'Non-Hispanic Black', 'Non-Hispanic White',
       'Non-Hispanic White,  younger than 20',
       'Total women aged 13-44 in need of contraceptive services and supplies',
       'Total women aged 13-44 in need of publicly funded contraceptive services and supplies',
       'Women aged 20-44 and <250% of federal poverty level in need of publicly funded contraceptive services and supplies',
       'Women younger than 20 in need of publicly funded contraceptive services and supplies',
       '% uninsured women aged 20-44 and <138% of federal poverty level in need of publicly funded contraceptive services',
       '% uninsured women aged 20-44 and between 138-249% of federal poverty level in need of publicly funded contraceptive services',
       '% uninsured women younger than 20 in need of publicly funded contraceptive services',
       'Total % uninsured women in need of publicly funded contraceptive services,',
       'Total uninsured women in need of publicly funded contraceptive services',
       'Uninsured women aged 20-44 and <138% of federal poverty level in need of publicly funded contraceptive services',
       'Uninsured women aged 20-44 and between 138-249% of federal poverty level  in need of publicly funded contraceptive services',
       'Uninsured women younger than 20 in need of publicly funded contraceptive services',
       'Number of Planned Parenthood clinics',
       'Number of federally funded qualified health centers',
       'Number of health department clinics', 'Number of hospitals',
       'Number of other clinics', 'Total publicly funded clinics',
       'Health department', 'Hospital', 'Other', 'Planned Parenthood',
       'Total Title X-funded clinics',
       'Number of female contraceptive clients served at Planned Parenthood clinics',
       'Number of female contraceptive clients served at federally funded qualified health centers',
       'Number of female contraceptive clients served at health department clinics',
       'Number of female contraceptive clients served at hospitals',
       'Number of female contraceptive clients served at other clinics',
       'Total', 'Total, younger than 20', 'Population of women aged 13-44',
       'Women aged 18-19 in need of contraceptive services and supplies',
       'Women aged 20-29 in need of contraceptive services and supplies',
       'Women aged 30-34 in need of contraceptive services and supplies',
       'Women younger than 18 in need of contraceptive services and supplies',
       'Total women aged 20-44 in need of contraceptive services and supplies  ',
       'Women aged 20-44 and <100% federal poverty level in need of contraceptive services and supplies',
       'Women aged 20-44 and >250% of poverty level in need of contraceptive services and supplies',
       'Women aged 20-44 and between 100-137% federal poverty level in need of contraceptive services and supplies',
       'Women aged 20-44 and between 138-199% federal poverty level in need of contraceptive services and supplies',
       'Women aged 20-44 and between 200-249% federal poverty level in need of contraceptive services and supplies','RAND_1', 'RAND_2', 'RAND_3', 'RAND_4', 'RAND_5']

Splitting into initial test-train sets

In [258]:
tp_tgt = df['High Risk']
tp_vars = df[feature_list]

In [259]:
### Converts the remaining non-integer features to integers

df['Total publicly funded clinics'] = df['Total publicly funded clinics'].astype(int)
df['Total'] = df['Total'].astype(int)
df['Total, younger than 20'] = df['Total, younger than 20'].astype(int)

In [260]:
### Replaces missing values in data with the median value

for i in feature_list:
    tp_vars[i] = tp_vars[i].replace(np.nan, tp_vars[i].median())

/Users/ramohse/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [261]:
### Train-test split

X_train, X_test, y_train, y_test = cross_validation.train_test_split(tp_vars, tp_tgt, test_size=0.3)

### Random Forest Model
Measures prediction accuracy and feature importance

In [262]:
max_features = len(feature_list)

rf =RandomForestClassifier(n_estimators = 200, max_features = max_features )
rf.fit(X_train, y_train)

a  = tp_vars.columns.values

ftr_imp = list(zip(a, rf.feature_importances_))

ftr_imp = sorted(ftr_imp, key=(lambda x: x[1]), reverse = True)

print(rf.score(X_test, y_test))
for i in ftr_imp:
    print(i)

0.860341151386
('% Children in Poverty', 0.33895936773792984)
('Chlamydia Rates per 100,000', 0.069046209705574735)
('% College', 0.04986841175370689)
('Avg Freshman Graduation Rate', 0.040101001642121929)
('Total % uninsured women in need of publicly funded contraceptive services,', 0.032539314910002726)
('Adherents %', 0.029544719517681756)
('% Single-Parent Households', 0.029008549227247587)
('% Obese', 0.017187070982098349)
('RAND_4', 0.016909839124059904)
('% Binge Drinking', 0.016646862056464571)
('RAND_1', 0.015508388028552549)
('RAND_5', 0.014674075865713376)
('Primary Care Physician Rate', 0.014504567727270028)
('GINI', 0.014470418742182736)
('RAND_3', 0.014120208973178519)
('% Smokers', 0.014019568718104579)
('RAND_2', 0.013849118756892445)
('% uninsured women younger than 20 in need of publicly funded contraceptive services', 0.013645324411312838)
('Congregations Per 10K People', 0.013563228661126268)
('Violent Crime Rate', 0.012497820210167123)
('Uninsured women aged 20-44 

### KNN (K-Nearest Neighbors) Model

In [263]:
knn_acc = []
rng = [x for x in range(1, 51)]

for n in rng:
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train, y_train)
    acc = accuracy_score(y_test, knn.predict(X_test))
    knn_acc.append(acc)
    print(n, acc)

1 0.670575692964
2 0.721748400853
3 0.7078891258
4 0.722814498934
5 0.712153518124
6 0.723880597015
7 0.726012793177
8 0.732409381663
9 0.727078891258
10 0.732409381663
11 0.727078891258
12 0.716417910448
13 0.722814498934
14 0.724946695096
15 0.719616204691
16 0.721748400853
17 0.721748400853
18 0.716417910448
19 0.717484008529
20 0.71855010661
21 0.720682302772
22 0.717484008529
23 0.715351812367
24 0.717484008529
25 0.717484008529
26 0.713219616205
27 0.714285714286
28 0.706823027719
29 0.711087420043
30 0.710021321962
31 0.713219616205
32 0.715351812367
33 0.717484008529
34 0.715351812367
35 0.71855010661
36 0.715351812367
37 0.717484008529
38 0.720682302772
39 0.720682302772
40 0.721748400853
41 0.722814498934
42 0.720682302772
43 0.721748400853
44 0.71855010661
45 0.717484008529
46 0.716417910448
47 0.715351812367
48 0.71855010661
49 0.716417910448
50 0.715351812367


### Logistic Regression Model

In [266]:
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
acc_lr = accuracy_score(y_test, log_reg.predict(X_test))

a  = tp_vars.columns.values
lr_coef = list(zip(a, log_reg.coef_[0]))
lr_coef = sorted(lr_coef, key=(lambda x: x[1]), reverse = True)

print('Logistic Regression accuracy: ', acc_lr)
for i in lr_coef:
    print(i)

Logistic Regression accuracy:  0.762260127932
('Chlamydia Rates per 100,000', 0.0030331528860133472)
('Women aged 20-44 and between 138-199% federal poverty level in need of contraceptive services and supplies', 0.0023887041300357328)
('Women aged 20-44 and between 100-137% federal poverty level in need of contraceptive services and supplies', 0.0010757448392296153)
('Women younger than 18 in need of contraceptive services and supplies', 0.00098391449910953149)
('Women aged 18-19 in need of contraceptive services and supplies', 0.00071108504185115847)
('Total, younger than 20', 0.00066707535387041891)
('Uninsured women aged 20-44 and between 138-249% of federal poverty level  in need of publicly funded contraceptive services', 0.00062140436818930987)
('Women aged 20-44 and between 200-249% federal poverty level in need of contraceptive services and supplies', 0.00039154141412292651)
('Hispanic', 0.0003759890656316058)
('Number of female contraceptive clients served at health department

In [268]:
### Quick and dirty model assessment, looking at the performance of six model types

models = {}

models['KNN'] = KNeighborsClassifier(n_neighbors=20)
models['LogReg'] = LogisticRegression()
models['GNB'] = GaussianNB()
models['DecTree'] = DecisionTreeClassifier()
models['RandForest'] = RandomForestClassifier(n_estimators = 100)
models['Gradient Trees'] = GradientBoostingClassifier()


score_list = ['accuracy', 'precision', 'recall', 'f1']

In [269]:
for name, model in models.items():
    print(name, ':')
    for m in score_list:
        scores = cv_score_means(model, tp_vars, tp_tgt, m)
        print(m, scores.mean())
    print('')

GNB :
accuracy 0.438295517859
precision 0.361290094453
recall 0.944355317885
f1 0.518383009999

DecTree :
accuracy 0.759121264388
precision 0.65195769679
recall 0.632679738562
f1 0.628673007697

RandForest :
accuracy 0.842562205863
precision 0.819259300857
recall 0.70101010101
f1 0.735541106604

Gradient Trees :
accuracy 0.84612788506
precision 0.806637690003
recall 0.737165775401
f1 0.752301223494

LogReg :
accuracy 0.761035361618
precision 0.728071154206
recall 0.426470588235
f1 0.508933464236

KNN :
accuracy 0.720381533246
precision 0.691368707832
recall 0.220617944147
f1 0.319693860839



## ROC Curves
Code for calculating and plotting ROC curves for models

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=23).fit(X_train, y_train)
GNB = GaussianNB().fit(X_train, y_train)
SVM = SVC(probability = True).fit(X_train, y_train)
RandForest = RandomForestClassifier(n_estimators = 15).fit(X_train, y_train)
DecTree = DecisionTreeClassifier().fit(X_train, y_train)
LogReg = LogisticRegression().fit(X_train, y_train)

KNN_pred = KNN.predict_proba(X_test)
GNB_pred = GNB.predict_proba(X_test)
SVM_pred = SVM.predict_proba(X_test)
RandForest_pred = RandForest.predict_proba(X_test)
DecTree_pred = DecTree.predict_proba(X_test)
LogReg_pred = LogReg.predict_proba(X_test)

In [ ]:
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, KNN_pred[:,1])
fpr_gnb, tpr_gnb, thresholds_gmb = roc_curve(y_test, GNB_pred[:,1])
fpr_svm, tpr_svm, thresholds_svm = roc_curve(y_test, SVM_pred[:,1])
fpr_randforest, tpr_randforest, thresholds_randforest = roc_curve(y_test, RandForest_pred[:,1])
fpr_dectree, tpr_dectree, thresholds_dectree = roc_curve(y_test, DecTree_pred[:,1])
fpr_logreg, tpr_logreg, thresholds_logreg = roc_curve(y_test, LogReg_pred[:,1])

In [ ]:
# row and column sharing
f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, sharex='col')
f.set_size_inches(18, 12)
ax1.plot(fpr_knn, tpr_knn);
ax1.plot(fpr_knn, fpr_knn, 'r--');
ax1.set_title('KNN');
ax2.plot(fpr_gnb, tpr_gnb);
ax2.plot(fpr_gnb, fpr_gnb, 'r--');
ax2.set_title('GNB');
ax3.plot(fpr_randforest, tpr_randforest);
ax3.plot(fpr_randforest, fpr_randforest, 'r--');
ax3.set_title('Random Forest');
ax4.plot(fpr_dectree, tpr_dectree);
ax4.plot(fpr_dectree, fpr_dectree, 'r--');
ax4.set_title('Decision Tree');
ax5.plot(fpr_logreg, tpr_logreg);
ax5.plot(fpr_logreg, fpr_logreg, 'r--');
ax5.set_title('Logistic Regression');
ax6.plot(fpr_svm, tpr_svm);
ax6.plot(fpr_svm, fpr_svm, 'r--');
ax6.set_title('SVM');